# Downloads

In [0]:
!git clone https://github.com/appinho/SAMaskRCNNKitti.git
%cd SAMaskRCNNKitti/
!wget https://github.com/matterport/Mask_RCNN/releases/download/v2.0/mask_rcnn_coco.h5
!wget https://s3.eu-central-1.amazonaws.com/avg-kitti/data_semantics.zip
!unzip -q data_semantics.zip

Cloning into 'SAMaskRCNNKitti'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (67/67), done.


# Mask R-CNN - Train on Shapes Dataset
This notebook shows how to train Mask R-CNN on the KITTI Instance Segmentation Dataset. To keep things simple we use only cars and pedestrians which enables fast training. You'd still need a GPU, though, because the network backbone is a Resnet101, which would be too slow to train on a CPU. On a GPU, you can start to get okay-ish results in a few minutes, and good results in less than an hour.

In [0]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt

from config import Config
import utils
import model as modellib
import visualize
from model import log

%matplotlib inline 

# Root directory of the project
ROOT_DIR = os.getcwd()

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Local path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

# Define image size with multiples of 64
IMG_HEIGHT = 320
IMG_WIDTH = 1152

# Configurations

In [0]:
class ShapesConfig(Config):
    """Configuration for training on the toy shapes dataset.
    Derives from the base Config class and overrides values specific
    to the toy shapes dataset.
    """
    # Give the configuration a recognizable name
    NAME = "shapes"

    # Train on 1 GPU and 8 images per GPU. We can put multiple images on each
    # GPU because the images are small. Batch size is 8 (GPUs * images/GPU).
    GPU_COUNT = 1
    IMAGES_PER_GPU = 8

    # Number of classes (including background)
    NUM_CLASSES = 1 + 2  # background + cars + pedestrians

    # Use small images for faster training. Set the limits of the small side
    # the large side, and that determines the image shape.
    IMAGE_MIN_DIM = min(IMG_WIDTH, IMG_HEIGHT)
    IMAGE_MAX_DIM = max(IMG_WIDTH, IMG_HEIGHT)
    
    # Learning rate
    LEARNING_RATE = 0.004 #0.002

    # Use smaller anchors because our image and objects are small
    RPN_ANCHOR_SCALES = (8, 16, 32, 64, 128)  # anchor side in pixels

    # Reduce training ROIs per image because the images are small and have
    # few objects. Aim to allow ROI sampling to pick 33% positive ROIs.
    TRAIN_ROIS_PER_IMAGE = 32

    # Use a small epoch since the data is simple
    STEPS_PER_EPOCH = 10

    # use small validation steps since the epoch is small
    VALIDATION_STEPS = 5
    
config = ShapesConfig(IMG_HEIGHT, IMG_WIDTH)
#config.IMAGE_SHAPE = np.array([self.IMAGE_MAX_DIM, self.IMAGE_MAX_DIM, 3])
print(config.IMAGE_SHAPE)


# Notebook Preferences

In [0]:
def get_ax(rows=1, cols=1, size=8):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Change the default size attribute to control the size
    of rendered images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

# Dataset
Create the KITTI dataset

Extend the Dataset class and add a method to load the KITTI dataset, load_road_participants(), and override the following methods:


* load_image()
* load_mask()
* image_reference()

In [0]:
import imageio

class ShapesDataset(utils.Dataset):
    """Generates the KITTI dataset. The dataset consists of simple
    road particispants (cars, pedestrians) placed within the scene.
    """
    
    def __init(self):
        self.counter = 0

    def load_road_participants(self, count, height, width):
        """Generate the requested number of synthetic images.
        count: number of images to generate.
        height, width: the size of the generated images.
        """
        # Add classes
        self.add_class("road_participants", 1, "car")
        self.add_class("road_participants", 2, "pedestrian")
        
        # Store height and width
        self.height = height
        self.width = width

        # Add images
        # Generate random specifications of images (i.e. color and
        # list of shapes sizes and locations). This is more compact than
        # actual images. Images are generated on the fly in load_image().
        for i in range(count):
            self.add_image("road_participants", image_id=i, 
                           path=None, width=width, height=height)
            #self.counter += 1

    def load_image(self, image_id):
        """Generate an image from the specs of the given image ID.
        Typically this function loads the image from a file, but
        in this case it generates the image on the fly from the
        specs in image_info.
        """
        info = self.image_info[image_id]
        image_name = str(info['id']).zfill(6) + "_10.png"
        image = imageio.imread("./training/image_2/" + image_name)
        image = cv2.resize(image, (self.width, self.height))
        if(image.shape[0] != self.height or image.shape[1] != self.width):
          print("WARNING: Strange shape after resizing image %d %d != %d %d" % 
                (image.shape[0], image.shape[1], self.height, self.width))
        return np.array(image)

    def image_reference(self, image_id):
        """Return the shapes data of the image."""
        info = self.image_info[image_id]
        if info["source"] == "shapes":
            return info["shapes"]
        else:
            super(self.__class__).image_reference(self, image_id)

    def load_mask(self, image_id):
        """Generate instance masks for shapes of the given image ID.
        """
        info = self.image_info[image_id]
        image_name = str(info['id']).zfill(6) + "_10.png"
        gt_image = imageio.imread("./training/instance/" + image_name)
        instance_gt = np.array(gt_image) % 256
        semantic_gt = np.array(gt_image) // 256
        instance_gt = cv2.resize(instance_gt, (self.width, self.height), interpolation=cv2.INTER_NEAREST)
        semantic_gt = cv2.resize(semantic_gt, (self.width, self.height), interpolation=cv2.INTER_NEAREST)
        labels = [26, 24]
        masks = []
        class_ids = []
        for l in labels:
            mask_sem = (semantic_gt == [l]).astype(np.int_) * 255
            mask_ins = instance_gt & mask_sem
            num_ins = np.max(mask_ins)
            if(num_ins > 30):
                print("WARNING: num ins %d for label l %d" % (num_ins, l))

            for i in range(1, num_ins + 1):
                mask_obj = (mask_ins == [i]).astype(np.int_) * 255
                masks.append(mask_obj)
                if l == 24:
                    class_ids.append(2)
                else:
                    class_ids.append(1)
        masks = np.array(masks)
        masks = np.moveaxis(masks, 0, -1)
        class_ids = np.array(class_ids)
        return masks, class_ids   

In [0]:
# Training dataset
dataset_train = ShapesDataset()
dataset_train.load_road_participants(1, config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1])
dataset_train.prepare()

# Validation dataset
dataset_val = ShapesDataset()
dataset_val.load_road_participants(1, config.IMAGE_SHAPE[0], config.IMAGE_SHAPE[1])
dataset_val.prepare()

In [0]:
# Load and display random samples
image_ids = np.random.choice(dataset_train.image_ids, 1)
for image_id in image_ids:
    print("image id", image_id)
    image = dataset_train.load_image(image_id)
    mask, class_ids = dataset_train.load_mask(image_id)
    print(image.shape, mask.shape, class_ids)
    visualize.display_top_masks(image, mask, class_ids, dataset_train.class_names)

# Evaluation before Transfer Learning

In [0]:
"""
# Create model in training mode
model = modellib.MaskRCNN(mode="inference", config=config,
                          model_dir=MODEL_DIR)

model_path = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

model.load_weights(model_path, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])

class InferenceConfig(ShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig(IMG_HEIGHT, IMG_WIDTH)

# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
#image_ids = np.random.choice(dataset_val.image_ids, 10)
image_ids = [i for i in range(0, 1)]
#print(image_ids)

APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox[:,:4], gt_bbox[:,4],
                         r["rois"], r["class_ids"], r["scores"])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))
"""

In [0]:
# Create model in training mode
model = modellib.MaskRCNN(mode="training", config=config,
                          model_dir=MODEL_DIR)

In [0]:
# Which weights to start with?
init_with = "coco"  # imagenet, coco, or last

if init_with == "imagenet":
    model.load_weights(model.get_imagenet_weights(), by_name=True)
elif init_with == "coco":
    # Load weights trained on MS COCO, but skip layers that
    # are different due to the different number of classes
    # See README for instructions to download the COCO weights
    model.load_weights(COCO_MODEL_PATH, by_name=True,
                       exclude=["mrcnn_class_logits", "mrcnn_bbox_fc", 
                                "mrcnn_bbox", "mrcnn_mask"])
elif init_with == "last":
    # Load the last model you trained and continue training
    model.load_weights(model.find_last(), by_name=True)

# Training
## Train in two stages:

1.   Only the heads. Here we're freezing all the backbone layers and training only the randomly initialized layers (i.e. the ones that we didn't use pre-trained weights from MS COCO). To train only the head layers, pass layers='heads' to the train() function.
2.   Fine-tune all layers. For this simple example it's not necessary, but we're including it to show the process. Simply pass layers="all" to train all layers.

In [0]:
# Train the head branches
# Passing layers="heads" freezes all layers except the head
# layers. You can also pass a regular expression to select
# which layers to train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE, 
            epochs=3, 
            layers='heads')

In [0]:
# Fine tune all layers
# Passing layers="all" trains all layers. You can also 
# pass a regular expression to select which layers to
# train by name pattern.
model.train(dataset_train, dataset_val, 
            learning_rate=config.LEARNING_RATE / 10,
            epochs=5, 
            layers="all")


In [0]:
# Save weights
# Typically not needed because callbacks save after every epoch
# Uncomment to save manually
# model_path = os.path.join(MODEL_DIR, "mask_rcnn_shapes.h5")
# model.keras_model.save_weights(model_path)

# Detection

In [0]:
class InferenceConfig(ShapesConfig):
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

inference_config = InferenceConfig(IMG_HEIGHT, IMG_WIDTH)

# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights
# model_path = os.path.join(ROOT_DIR, ".h5 file name here")
model_path = model.find_last()[1]

# Load trained weights (fill in path to trained weights here)
assert model_path != "", "Provide path to trained weights"
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

In [0]:
# Test on a random image
image_id = random.choice(dataset_val.image_ids)
original_image, image_meta, gt_bbox, gt_mask =\
    modellib.load_image_gt(dataset_val, inference_config, 
                           image_id, use_mini_mask=False)

log("original_image", original_image)
log("image_meta", image_meta)
log("gt_bbox", gt_bbox)
log("gt_mask", gt_mask)

visualize.display_instances(original_image, gt_bbox[:,:4], gt_mask, gt_bbox[:,4], 
                            dataset_train.class_names, figsize=(8, 8))

In [0]:
results = model.detect([original_image], verbose=1)
r = results[0]
print(r['rois'][0], r['masks'][0].shape)
visualize.display_instances(original_image, r['rois'], r['masks'], r['class_ids'], 
                            dataset_val.class_names, r['scores'], ax=get_ax())

# Evaluation after Transfer Learning

In [0]:
# Compute VOC-Style mAP @ IoU=0.5
# Running on 10 images. Increase for better accuracy.
#image_ids = np.random.choice(dataset_val.image_ids, 10)
image_ids = [i for i in range(0, 1)]
#print(image_ids)

APs = []
for image_id in image_ids:
    # Load image and ground truth data
    image, image_meta, gt_bbox, gt_mask =\
        modellib.load_image_gt(dataset_val, inference_config,
                               image_id, use_mini_mask=False)
    molded_images = np.expand_dims(modellib.mold_image(image, inference_config), 0)
    # Run object detection
    results = model.detect([image], verbose=0)
    r = results[0]
    # Compute AP
    AP, precisions, recalls, overlaps =\
        utils.compute_ap(gt_bbox[:,:4], gt_bbox[:,4],
                         r["rois"], r["class_ids"], r["scores"])
    APs.append(AP)
    
print("mAP: ", np.mean(APs))